In [1]:
import csv
import pandas as pd
import numpy as np
from time import time
from copy import deepcopy

In [2]:
## Données
d_categories={"Bureaux Administration":0,
"Café Hôtel Restaurant":1,
"Commerce":2,
"Enseignement Recherche":3,
"Habitat Communautaire":4,
"Santé Action Sociale":5,
"Sport Loisir Culture":6,
"Transport":7}

L_categories=["Bureaux Administration",
"Café Hôtel Restaurant",
"Commerce",
"Enseignement Recherche",
"Habitat Communautaire",
"Santé Action Sociale",
"Sport Loisir Culture",
"Transport"]

L_surfaces=[223560000,68040000,213840000,184680000,68040000,
116640000,68040000,29160000]

d_type_energie={"Biomasse":0,
"Chaudière fioul":1,
"Chaudière gaz":2,
"Chauffage urbain":3,
"Chauffage électrique":4,
"Pompes à chaleur air-eau":5,
"Pompes à chaleur air-air":6,
"Pompes à chaleur hybride":7}

L_type_energie=["Biomasse","Chaudière fioul","Chaudière gaz",
"Chauffage urbain","Chauffage électrique",
"Pompes à chaleur air-eau","Pompes à chaleur air-air","Pompes à chaleur hybride"]
L_energie=[[0.02,0.12,0.43,0.08,0.23,0.12,0,0],
           [0.02,0.18,0.35,0.06,0.26,0.13,0,0],
           [0.05,0.22,0.38,0.03,0.21,0.05,0.06,0],
           [0.04,0.21,0.59,0.06,0.07,0.03,0,0],
           [0.06,0.27,0.45,0.05,0.11,0.06,0,0],
           [0.02,0.19,0.56,0.08,0.1,0.05,0,0],
           [0.09,0.15,0.41,0.07,0.19,0.09,0,0],
           [0.04,0.22,0.45,0.05,0.16,0.04,0.04,0]]

L_lettre=["A","B","C","D","E","F","G"]
L_classe_energie=[[0,0,0.2,0.47,0.25,0.08,0],
                  [0,0,0.22,0.24,0.29,0.22,0.03],
                  [0,0.005,0.24,0.52,0.22,0.015,0],
                  [0.005,0.14,0.59,0.23,0.03,0.005,0],
                  [0,0.26,0.53,0.2,0.01,0,0],
                  [0,0.1,0.27,0.55,0.07,0.01,0],
                  [0,0.01,0.08,0.33,0.25,0.13,0.2],
                  [0,0,0,0,0.06,0.94,0]]
d_lettre_energie={"A":41,"B":69,"C":125,"D":191,"E":276,"F":380,"G":546}

In [3]:
## Constitution de la base

L_base_index=[]
L_base_var=[]
for i in range(8):
    for j in range(8):
        for k in range(7):
                L_base_index.append([L_categories[i],L_type_energie[j],L_lettre[k],"old"])
                L_base_var.append([round(L_surfaces[i]*L_energie[i][j]*L_classe_energie[i][k]),
                               d_lettre_energie[L_lettre[k]]])

In [4]:
## Calcul transitions chauffage
L_trans_all=[[[1,0,0,0,0,0,0,0],
[0.02,0,0,0.2,0,0.78,0,0],
[0.02,0,0.05,0.2,0,0.73,0,0],
[0,0,0,1,0,0,0,0],
[0.02,0,0,0.2,0.2,0,0.58,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1]],

[[1,0,0,0,0,0,0,0],
[0.02,0,0,0.15,0,0.71,0.12,0],
[0.02,0,0.5,0.15,0,0.29,0.04,0],
[0,0,0,1,0,0,0,0],
[0.02,0,0,0.15,0.5,0,0.33,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1]],

[[1,0,0,0,0,0,0,0],
[0.05,0,0,0.15,0,0.72,0.08,0],
[0.05,0,0.4,0.15,0,0.36,0.04,0],
[0,0,0,1,0,0,0,0],
[0.05,0,0,0.15,0.3,0,0.5,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1]],

[[1,0,0,0,0,0,0,0,0],
[0.04,0,0,0.15,0,0.65,0.16,0],
[0.04,0,0.05,0.15,0,0.61,0.15,0],
[0,0,0,1,0,0,0,0],
[0.04,0,0,0.15,0.1,0,0.71,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1]],

[[1,0,0,0,0,0,0,0],
[0.06,0,0,0.15,0,0.71,0.08,0],
[0.06,0,0.05,0.15,0,0.64,0.07,0],
[0,0,0,1,0,0,0,0],
[0.06,0,0,0.15,0.2,0,0.59,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1]],
             
[[1,0,0,0,0,0,0,0],
[0.02,0,0,0.1,0,0.79,0.09,0],
[0.02,0,0.05,0.1,0,0.75,0.08,0],
[0,0,0,1,0,0,0,0],
[0.02,0,0,0.1,0.1,0,0.78,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1]],

[[1,0,0,0,0,0,0,0],
[0.09,0,0,0.1,0,0.73,0.08,0],
[0.09,0,0.05,0.1,0,0.68,0.08,0],
[0,0,0,1,0,0,0,0],
[0.09,0,0,0.1,0.2,0,0.61,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1]],

[[1,0,0,0,0,0,0,0],
[0.04,0,0,0.2,0,0.76,0,0],
[0.04,0,0.05,0.2,0,0.71,0,0],
[0,0,0,1,0,0,0,0],
[0.04,0,0,0.2,0.1,0,0.66,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1]]]


L_heat_all=[[0 for i in range(8)] for j in range(8)]
for i in range(len(L_base_index)):
    L_heat_all[d_categories[L_base_index[i][0]]][d_type_energie[L_base_index[i][1]]]+=L_base_var[i][0]
        
S_trans_all=[[[L_heat_all[i][j]*L_trans_all[i][j][k] for k in range(8)] for j in range(8)] for i in range(8)]

In [5]:
### Modèle complet d'évolution du parc

## Fin de vie des bâtiments
taux_disp=0.005# taux de disparition annuel

def survie_global(L_base_index,L_base_var):
    L_base_index_return=deepcopy(L_base_index)
    L_base_var_return=deepcopy(L_base_var)
    for i in range(len(L_base_index)):
        if L_base_index[i][-1]=="old":
            L_base_var_return[i][0]=round(L_base_var_return[i][0]*(1-taux_disp))
    return L_base_index_return,L_base_var_return

## Rénovation
d_efficiency_max={"Bureaux Administration":0.63,
"Café Hôtel Restaurant":0.25,
"Commerce":0.41,
"Enseignement Recherche":0.57,
"Habitat Communautaire":0.25,
"Santé Action Sociale":0.43,
"Sport Loisir Culture":0.47,
"Transport":0.25}
year_ref=2020
year_optim=2030
efficiency_start=0.15
def reno(categorie,energie,classe_energie,year):
    # Ne rénove pas les classes A ou B 
    if classe_energie in ["A","B"]:
        return energie
    else:
        if year>=year_optim:
            return energie*(1-d_efficiency_max[categorie])
        else:
            return energie*(1-(efficiency_start+(year-year_ref)/(year_optim-year_ref))*(d_efficiency_max[categorie]-efficiency_start))

def energy_to_class(energy):
    if energy<=50:
        return "A"
    elif energy<=90:
        return "B"
    elif energy<=150:
        return "C"
    elif energy<=230:
        return "D"
    elif energy<=330:
        return "E"
    elif energy<=450:
        return "F"
    else:
        return "G"

taux_2020=0.015
taux_2030=0.02
taux_2040=0.025
taux_2050=0.025
taux_2060=0.025
def taux_reno(year):
    if year<=2030:
        return taux_2020+(year-2020)/10*(taux_2030-taux_2020)
    elif year<=2040:
        return taux_2030+(year-2030)/10*(taux_2040-taux_2030)
    elif year<=2050:
        return taux_2040+(year-2040)/10*(taux_2050-taux_2040)
    else:
        return taux_2050+(year-2050)/10*(taux_2060-taux_2050)

d_letter={'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6}
L_letter=['A','B','C','D','E','F','G']
def renovation_global(L_base_index,L_base_var,year):
    d_surf_class={'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0}
    for i in range(len(L_base_index)):
        d_surf_class[L_base_index[i][2]]+=L_base_var[i][0]
    
    p_reno_class={'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0}
    L_cumul_surf_class=[0 for i in range(7)]
    for i in range(7):
        L_cumul_surf_class[i]=L_cumul_surf_class[i-1]+d_surf_class[L_letter[-i-1]]
    surf_reno=taux_reno(year)*L_cumul_surf_class[-1]
    i_stop=0
    b_continue=True
    while b_continue and i_stop<7:
        if surf_reno>=L_cumul_surf_class[i_stop]:
            i_stop+=1
        else:
            b_continue=False
    L_cumul_surf_class=[0]+L_cumul_surf_class
    for i in range(i_stop):
        p_reno_class[L_letter[-i-1]]=1
    p_reno_class[L_letter[-i_stop-1]]=(surf_reno-L_cumul_surf_class[i_stop])/d_surf_class[L_letter[-i_stop-1]]
    L_base_index_return=deepcopy(L_base_index)
    L_base_var_return=deepcopy(L_base_var)
    
    for i in range(len(L_base_index)):
        S_renov=round(L_base_var_return[i][0]*p_reno_class[L_base_index[i][2]])
        if S_renov>0:
            L_base_var_return[i][0]-=S_renov
            new_energy=reno(L_base_index[i][0],L_base_var_return[i][1],L_base_index[i][2],year)
            new_class=energy_to_class(new_energy)
            New_index=[L_base_index[i][0],L_base_index[i][1],new_class,L_base_index[i][-1]]
            i_new=L_base_index_return.index(New_index)
            surf_rec=L_base_var_return[i_new][0]
            energy_rec=L_base_var_return[i_new][1]
            energy_mean=(surf_rec*energy_rec+S_renov*new_energy)/(surf_rec+S_renov)
            L_base_var_return[i_new][0]+=S_renov
            L_base_var_return[i_new][1]=energy_mean
    return L_base_index_return,L_base_var_return

## Changement mode de chauffage
year_ref_trans=2020
year_optim_trans=2030
year_goal_trans=2050

year_optim_fioul=2024
year_goal_fioul=2030# Le gouvernement viserait la sortie complète du fioul en 2028 (PPE). 
#En plus cela aide à tenir l'objectif fit-for-55.

def mat_trans(L_base_index,L_base_var,year):
    P_trans_all=[[[0 for i in range(8)] for j in range(8)] for k in range(8)]
    
    L_heat_all_t=[[0 for i in range(8)] for j in range(8)]
    for i in range(len(L_base_index)):
        L_heat_all_t[d_categories[L_base_index[i][0]]][d_type_energie[L_base_index[i][1]]]+=L_base_var[i][0]
    if year<=year_optim_fioul:
        for k in range(8):
            for i in [0,2,3,4,5,6,7]:
                for j in range(8):
                    P_trans_all[k][i][j]=S_trans_all[k][i][j]/(year_goal_trans-(year_ref_trans+year_optim_trans)/2+1/2)\
                    *(year-year_ref_trans)/(year_optim_trans-year_ref_trans)
            for j in range(8):
                P_trans_all[k][1][j]=S_trans_all[k][1][j]/(year_goal_fioul-(year_ref_trans+year_optim_fioul)/2+1/2)\
                *(year-year_ref_trans)/(year_optim_fioul-year_ref_trans)
    elif year<=year_optim_trans:
        for k in range(8):
            for i in [0,2,3,4,5,6,7]:
                for j in range(8):
                    P_trans_all[k][i][j]=S_trans_all[k][i][j]/(year_goal_trans-(year_ref_trans+year_optim_trans)/2+1/2)\
                    *(year-year_ref_trans)/(year_optim_trans-year_ref_trans)
            for j in range(8):
                P_trans_all[k][1][j]=S_trans_all[k][1][j]/(year_goal_fioul-(year_ref_trans+year_optim_fioul)/2+1/2)
    else:
        for k in range(8):
            for i in [0,2,3,4,5,6,7]:
                for j in range(8):
                    P_trans_all[k][i][j]=S_trans_all[k][i][j]/(year_goal_trans-(year_ref_trans+year_optim_trans)/2+1/2)
            for j in range(8):
                P_trans_all[k][1][j]=S_trans_all[k][1][j]/(year_goal_fioul-(year_ref_trans+year_optim_fioul)/2+1/2)
    
    for k in range(8):
        for i in range(8):
            denom_k=max(L_heat_all_t[k][i],sum(P_trans_all[k][i]),1)        
            for j in range(8):
                P_trans_all[k][i][j]=P_trans_all[k][i][j]/denom_k
            
    return P_trans_all

def heat_trans(L_base_index,L_base_var,year):
    L_base_index_return=deepcopy(L_base_index)
    L_base_var_return=deepcopy(L_base_var)
    
    P_trans_all=mat_trans(L_base_index,L_base_var,year)
    
    for i in range(len(L_base_index)):
        L_trans=P_trans_all[d_categories[L_base_index[i][0]]][d_type_energie[L_base_index[i][1]]]
        S_trans=[]
        for j in range(8):
            if j!=d_type_energie[L_base_index[i][1]]:
                S_trans.append(round(L_base_var_return[i][0]*L_trans[j]))
            else:
                S_trans.append(0)
        while sum(S_trans)>L_base_var_return[i][0]:
            # Il arrive que l'on transitionne plus de surface qu'il y en a...
            j_max=S_trans.index(max(S_trans))
            S_trans[j_max]-=1
        
        for j in range(8):
            if S_trans[j]>0:
                New_index=[L_base_index[i][0],L_type_energie[j],L_base_index[i][2],L_base_index[i][-1]]
                try:
                    i_new=L_base_index_return.index(New_index)
                    S_rec=L_base_var_return[i_new][0]
                    energy_rec=L_base_var_return[i_new][1]
                    energy_i=L_base_var_return[i][1]
                    energy_mean=(S_rec*energy_rec+S_trans[j]*energy_i)/(S_rec+S_trans[j])
                    L_base_var_return[i_new][0]+=S_trans[j]
                    L_base_var_return[i_new][1]=energy_mean
                except:
                    L_base_index_return.append(New_index)
                    L_base_var_return.append([S_trans[j],L_base_var_return[i][1]])
        L_base_var_return[i][0]-=sum(S_trans)
        
    
    return L_base_index_return,L_base_var_return

## Neuf
L_heat_new_all0=[[0.02,0,0.19,0.04,0.08,0.61,0.06,0],
                 [0.02,0,0.22,0.02,0.2,0.43,0.11,0],
                 [0.04,0,0.54,0.09,0.07,0.21,0.05,0],
                 [0.01,0,0.15,0,0.19,0.48,0.17,0],
                 [0.02,0,0.22,0.02,0.2,0.43,0.11,0],
                 [0.02,0,0.22,0.02,0.2,0.43,0.11,0],
                 [0.02,0,0.22,0.02,0.2,0.38,0.16,0],
                 [0.02,0,0.22,0.02,0.2,0.36,0.18,0]]
L_heat_new_all=[[0.03,0,0,0.2,0.02,0.68,0.07,0],
                [0.02,0,0,0.05,0.1,0.67,0.16,0],
                [0.04,0,0,0.2,0.05,0.56,0.15,0],
                [0.02,0,0,0.1,0.1,0.58,0.2,0],
                [0.02,0,0,0.2,0,0.63,0.15,0],
                [0.02,0,0,0.2,0,0.63,0.15,0],
                [0.15,0,0,0.2,0,0.45,0.2,0],
                [0.04,0,0,0.2,0,0.52,0.24,0]]

year_new_all=2026#Jusqu'à fin 2024 dans les bâtiments tertiaires il est possible d'installer du gaz
# Une marge de 2 ans pour le permis de construire est considérée.

S_new_year=[1730417,
526649,
1655181,
1429475,
526649,
902826,
526649,
225707]
energy_new=50

def add_new(L_base_index,L_base_var,year):
    L_base_index_return=deepcopy(L_base_index)
    L_base_var_return=deepcopy(L_base_var)
    
    for i in range(8):#Categorie
        for j in range(8):#Mode de chauffage
            New_index=[L_categories[i],L_type_energie[j],"B","new"]
            if year>=year_new_all:
                S=round(L_heat_new_all[i][j]*S_new_year[i])
            else:
                S=round(L_heat_new_all0[i][j]*S_new_year[i])                
            if S>0:
                try:
                    i_new=L_base_index_return.index(New_index)
                    S_rec=L_base_var_return[i_new][0]
                    energy_rec=L_base_var_return[i_new][1]
                    energy_mean=(S_rec*energy_rec+S*energy_new)/(S_rec+S)                        
                    L_base_var_return[i_new][0]+=S
                    L_base_var_return[i_new][1]=energy_mean
                except:
                    L_base_index_return.append(New_index)
                    L_base_var_return.append([S,energy_new])
    
    return L_base_index_return,L_base_var_return

In [6]:
## Tests
t1=time()
L_base_index_test,L_base_var_test=survie_global(L_base_index,L_base_var)
L_base_index_test,L_base_var_test=renovation_global(L_base_index_test,L_base_var_test,2021)
L_base_index_test,L_base_var_test=heat_trans(L_base_index_test,L_base_var_test,2021)
L_base_index_test,L_base_var_test=add_new(L_base_index_test,L_base_var_test,2021)
t2=time()
print(t2-t1)

0.03098011016845703


In [7]:
## Simultion 2020-2060
Taux_heat=[0.682,0.577,0.533,0.735,0.923,0.776,0.514,0.57]# Calculé sur données initiales

L_base_indexi,L_base_vari=deepcopy(L_base_index),deepcopy(L_base_var)
S_heat=[[0 for i in range(8)]]
E_heat=[[0 for i in range(8)]]
S_class=[[0 for i in range(7)]]

for i in range(len(L_base_indexi)):
    j=d_type_energie[L_base_indexi[i][1]]
    S_heat[-1][j]+=L_base_vari[i][0]
    E_heat[-1][j]+=L_base_vari[i][0]*L_base_vari[i][1]*Taux_heat[d_categories[L_base_indexi[i][0]]]
    S_class[-1][d_letter[L_base_indexi[i][2]]]+=L_base_vari[i][0]
    
for year in range(2021,2061):
    print(year)
    if year<=2050:
        L_base_indexi,L_base_vari=survie_global(L_base_indexi, L_base_vari)
        L_base_indexi,L_base_vari=renovation_global(L_base_indexi, L_base_vari, year)
        L_base_indexi,L_base_vari=heat_trans(L_base_indexi, L_base_vari, year)
        L_base_indexi,L_base_vari=add_new(L_base_indexi, L_base_vari, year)
    else:
        L_base_indexi,L_base_vari=survie_global(L_base_indexi, L_base_vari)
        L_base_indexi,L_base_vari=renovation_global(L_base_indexi, L_base_vari, year)
        L_base_indexi,L_base_vari=add_new(L_base_indexi, L_base_vari, year)
    S_heat.append([0 for i in range(8)])
    E_heat.append([0 for i in range(8)])
    S_class.append([0 for i in range(7)])

    for i in range(len(L_base_indexi)):
        j=d_type_energie[L_base_indexi[i][1]]
        S_heat[-1][j]+=L_base_vari[i][0]
        E_heat[-1][j]+=L_base_vari[i][0]*L_base_vari[i][1]*Taux_heat[d_categories[L_base_indexi[i][0]]]
        S_class[-1][d_letter[L_base_indexi[i][2]]]+=L_base_vari[i][0]

filename="Surface_tertiaire_type_de_chauffage.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_type_energie)
    for year in range(2020,2061):
        fwrite.writerow([year]+S_heat[year-2020])
filename="Energie_tertiaire_type_de_chauffage.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_type_energie)
    for year in range(2020,2061):
        fwrite.writerow([year]+[x/1e9 for x in E_heat[year-2020]])
filename="Surface_tertiaire_classe_energie.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_lettre)
    for year in range(2020,2061):
        fwrite.writerow([year]+S_class[year-2020])

2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [8]:
## Résultats 2060
surface=0
energy=0
for i in range(len(L_base_vari)):
    surface+=L_base_vari[i][0]
    energy+=L_base_vari[i][1]*L_base_vari[i][0]
print(surface)
print(energy)
print(energy/surface)
#d_corr_class={'A':1.3,'B':1.1,'C':0.85,'D':0.75,'E':0.65,'F':0.6,'G':0.55}
d_surf_class={'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0}
d_energy_class={'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0}
for i in range(len(L_base_indexi)):
    d_surf_class[L_base_indexi[i][2]]+=L_base_vari[i][0]
    d_energy_class[L_base_indexi[i][2]]+=L_base_vari[i][1]*L_base_vari[i][0]
L_results=[[d_surf_class[L_letter[i]] for i in range(7)],
          [d_energy_class[L_letter[i]] for i in range(7)]]
print(L_results)

d_categories={"Bureaux Administration":0,"Café Hôtel Restaurant":1,"Commerce":2,"Enseignement Recherche":3,
              "Habitat Communautaire":4,"Santé Action Sociale":5,"Sport Loisir Culture":6,"Transport":7}
L_results2=[[0 for i in range(8)],[0 for i in range(8)]]
for i in range(len(L_base_indexi)):
    L_results2[0][d_categories[L_base_indexi[i][0]]]+=L_base_vari[i][0]
    L_results2[1][d_categories[L_base_indexi[i][0]]]+=L_base_vari[i][1]*L_base_vari[i][0]
print(L_results2)

1096349364
98320503198.59589
89.67990170567188
[[8478079, 553714146, 534157139, 0, 0, 0, 0], [350227315.3910833, 33965663977.99937, 64004611905.205444, 0.0, 0.0, 0.0, 0.0]]
[[252160324, 76744463, 241196781, 208306443, 76744475, 131561912, 76744474, 32890492], [19666143880.0259, 8908050704.009834, 22188964646.72184, 18411076769.31211, 7308638183.152903, 11748807489.231924, 6161656476.382476, 3927165049.7589245]]


In [9]:
## État des lieux 2020
surface=0
energy=0
for i in range(len(L_base_var)):
    surface+=L_base_var[i][0]
    energy+=L_base_var[i][1]*L_base_var[i][0]
print(surface)
print(energy)
print(energy/surface)
#d_corr_class={'A':1.3,'B':1.1,'C':0.85,'D':0.75,'E':0.65,'F':0.6,'G':0.55}
d_surf_class={'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0}
d_energy_class={'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0}
for i in range(len(L_base_index)):
    d_surf_class[L_base_index[i][2]]+=L_base_var[i][0]
    d_energy_class[L_base_index[i][2]]+=L_base_var[i][1]*L_base_var[i][0]
L_results=[[d_surf_class[L_letter[i]] for i in range(7)],
          [d_energy_class[L_letter[i]] for i in range(7)]]
print(L_results)

d_categories={"Bureaux Administration":0,"Café Hôtel Restaurant":1,"Commerce":2,"Enseignement Recherche":3,
              "Habitat Communautaire":4,"Santé Action Sociale":5,"Sport Loisir Culture":6,"Transport":7}
L_results2=[[0 for i in range(8)],[0 for i in range(8)]]
for i in range(len(L_base_index)):
    L_results2[0][d_categories[L_base_index[i][0]]]+=L_base_var[i][0]
    L_results2[1][d_categories[L_base_index[i][0]]]+=L_base_var[i][1]*L_base_var[i][0]
print(L_results2)

972000000
192091354200
197.62485
[[923400, 56959200, 292960800, 375289200, 155811600, 74406600, 15649200], [37859400, 3930184800, 36620100000, 71680237200, 43004001600, 28274508000, 8544463200]]
[[223560000, 68040000, 213840000, 184680000, 68040000, 116640000, 68040000, 29160000], [47879845200, 17238614400, 41930816400, 25435053000, 8515206000, 19691164800, 20501812800, 10898841600]]
